In [1]:
from caveclient import CAVEclient
from cloudvolume import CloudVolume
from meshparty import trimesh_vtk
import trimesh

In [2]:
cv = CloudVolume("precomputed://gs://iarpa_microns/minnie/minnie65/seg_m1300", use_https=True)

In [3]:

mesh = cv.mesh.get(864691136194411734, lod=2)[864691136194411734]
mesh_actor = trimesh_vtk.mesh_actor(mesh,
                                    color=(1,0,0),
                                    opacity=0.5)
mesh_obj=(mesh.to_obj())
type(mesh_obj)

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.26it/s]


bytes

In [4]:
print(mesh.vertices.shape, mesh.faces.shape)

(31237, 3) (58510, 3)


In [ ]:
def mesh_to_binary_volume(mesh, pitch: float = 1.0):
    """
    Convert cloudvolume Mesh into a 3D binary numpy array.
    
    Parameters
    ----------
    mesh : cloudvolume.mesh.Mesh
        The mesh object from CloudVolume.
    pitch : float
        The voxel size (resolution) in the same units as the mesh coordinates.
    
    Returns
    -------
    voxels : np.ndarray
        3D boolean array, True = inside the mesh.
    transform : np.ndarray
        Affine transform to map voxel indices back to world coordinates.
    """
    # Convert CloudVolume mesh into trimesh
    tm = trimesh.Trimesh(vertices=mesh.vertices, faces=mesh.faces, process=False)

    # Voxelize mesh at given pitch
    v = tm.voxelized(pitch=pitch)

    # Export dense boolean matrix
    voxels = v.matrix.astype(bool)

    # Affine transform to world space
    transform = v.transform  

    return voxels, transform

binary_mask, transform = mesh_to_binary_volume(mesh, pitch=0.2)

print(binary_mask.shape, binary_mask.dtype)  # e.g. (100, 120, 80) bool
